<a href="https://colab.research.google.com/github/Henil21/Natural_Language_processing/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals in Tensorflow

The main goal of natural language processing (NLP) is to derive information from natural language.

Natural language is a broad term but you can consider it to cover any of the following:

* Text (such as that contained in an email, blog post, book, Tweet)
* Speech (a conversation you have with a doctor, voice commands you give to a smart speaker)


> Text -> turn into numbers -> build a model -> train the model to find patterns -> use patterns (make predictions)


In [1]:
!nvidia-smi  -L

GPU 0: Tesla T4 (UUID: GPU-a61bb77e-83bf-6099-8965-cc7bf931e986)


## getting helper functions 🐚

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-07-16 06:12:26--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-07-16 06:12:27 (59.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# importing series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves,compare_historys

## Get a text dataset

>description of data set: text sample of tweet labelled as disaster or not disaster.

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')

--2023-07-16 06:12:31--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.128, 142.250.31.128, 142.251.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-07-16 06:12:31 (128 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing Our Data


In [5]:
import pandas as pd
train_dir=pd.read_csv("train.csv")
test_dir=pd.read_csv("test.csv")
train_dir.head()


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# shuffling the training data
train_shf=train_dir.sample(frac=1,random_state=42)
train_shf.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_dir.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# lets visualize some random training example
import random
random_index=random.randint(0, len(train_dir)-5)
# create random index not higher than total number of samples
for row in train_shf[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"target:{target}","(real disaster)" if target>0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("----\n")

target:1 (real disaster)
Text:
Suicide Bomber Kills 13 At Saudi Mosque http://t.co/oZ1DS3Xu0D #Saudi #Tripoli #Libya

----

target:0 (not real disaster)
Text:
People are finally panicking about cable TV http://t.co/lCnW4EAD8v

----

target:1 (real disaster)
Text:
Map: Typhoon Soudelor's predicted path as it approaches Taiwan; expected to make landfall over southern China by... http://t.co/YvaFI3zuJx

----

target:0 (not real disaster)
Text:
Guns are for protection.. 
That shit really shouldn't be used unless your life in danger

----

target:0 (not real disaster)
Text:
#3: TITAN WarriorCord 100 Feet - Authentic Military 550 Paracord - MIL-C-5040-H Type III 7 Strand 5/16' di... http://t.co/EEjRMKtJ0R

----



### Split data into training and validation sets ✅

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# use train_test_split to split training data into training and validation sets
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_shf["text"].to_numpy(),
                                                                       train_shf["target"].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=42)

In [11]:
val_sentences[:10]
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [12]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [13]:
val_labels.shape

(762,)

### Converting Text into number ⚡




* Tokenization - A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization:
1. Using word-level tokenization with the sentence "I love TensorFlow" might result in "I" being 0, "love" being 1 and "TensorFlow" being 2. In this case, every word in a sequence considered a single token.
2. Character-level tokenization, such as converting the letters A-Z to values 1-26. In this case, every character in a sequence considered a single token.
3. Sub-word tokenization is in between word-level and character-level tokenization. It involves breaking invidual words into smaller parts and then converting those smaller parts into numbers. For example, "my favourite food is pineapple pizza" might become "my, fav, avour, rite, fo, oo, od, is, pin, ine, app, le, piz, za". After doing this, these sub-words would then be mapped to a numerical value. In this case, every word could be considered multiple tokens.
* Embeddings - An embedding is a representation of natural language which can be learned. Representation comes in the form of a feature vector. For example, the word "dance" could be represented by the 5-dimensional vector [-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]. It's important to note here, the size of the feature vector is tuneable. There are two ways to use     embeddings:
1. Create your own embedding - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as tf.keras.layers.Embedding) and an embedding representation will be learned during model training.
2. Reuse a pre-learned embedding - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.

### Text vectorization (Tokenization)
* The TextVectorization layer takes the following parameters:

*  max_tokens - The maximum number of words in your vocabulary (e.g. 20000 or the number of unique words in your text), includes a value for OOV (out of vocabulary) tokens.
* standardize - Method for standardizing text. Default is "lower_and_strip_punctuation" which lowers text and removes all punctuation marks.
* split - How to split text, default is "whitespace" which splits on spaces.
* ngrams - How many words to contain per token split, for example, ngrams=2 splits tokens into continuous sequences of 2.
* output_mode - How to output tokens, can be "int" (integer mapping), "binary" (one-hot encoding), "count" or "tf-idf". See documentation for more.
* output_sequence_length - Length of tokenized sequence to output. For example, if output_sequence_length=150, all tokenized sequences will be 150 tokens long.
* pad_to_max_tokens - Defaults to False, if True, the output feature axis will be padded to max_tokens even if the number of unique tokens in the vocabulary is less than max_tokens. Only valid in certain modes, see docs for more.

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#using default textvectorization parameters
text_vec=TextVectorization(max_tokens=None, #how many different words are in our vocabaulary (automatically add <ODV>)
                           standardize="lower_and_strip_punctuation",#convert all to lower case and remove punctucations as it doesnot contribute much in output
                           split="whitespace",
                           ngrams=None, #create groupe of n-word,
                           output_mode="int",
                           output_sequence_length=None,#how long we want our sequences to be(how long a tweet can be)
                          #  pad_to_max_tokens=True [not valid if max_token is set to None]
                           )


In [15]:
len(train_sentences[0].split())

7

In [16]:
# find the average number of token(words) in the training tweet
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                     split="whitespace",
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [18]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

# create a smple sentence and tokenize it
sample_tweet="There's a flood in my street"
text_vectorizer([sample_tweet])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# choose a random sentence from training dataset and tokenizing the,
random_sentence=random.choice(train_sentences)
print(f"original text:\n{random_sentence}\n\n vectorized text",text_vectorizer(random_sentence))

original text:
UNPREDICTABLE DISCONNECTED AND SOCIAL CASUALTY ARE MY FAVORITES HOW DO PEOPLE NOT LIKE THEM

 vectorized text tf.Tensor(
[7183    1    7 1761  712   22   13 5662   62   68   57   34   25   93
    0], shape=(15,), dtype=int64)


In [20]:
# Get the unique words in vocabulary
words_in_voc=text_vectorizer.get_vocabulary()
top_5=words_in_voc[:5]
bottom_5=words_in_voc[-5:]
print(f"number of word in vocab {len(words_in_voc)}\n")
print(f"5 most comman word in  vocab {top_5}\n")
print(f"5 least comman  word in vocab {bottom_5}")


number of word in vocab 10000

5 most comman word in  vocab ['', '[UNK]', 'the', 'a', 'in']

5 least comman  word in vocab ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding Layer

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The Parameters we care most about are
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, eg:- a value of 100 would mean each token get represented by a vector of 100 long
* `input_length` = The length of the sequences being passed to the embedding layer

In [21]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,#set the input shape
                             output_dim=128,
                             input_length=max_length)

In [22]:
# Get random sentence
random_sentence=random.choice(train_sentences)
print(f"orignal text:\n{random_sentence}")

# embed the random sentence (turn it into dense vector of the fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))



orignal text:
#news Britons rescued amid Himalaya floods http://t.co/kEPznhXHXd


In [23]:
#  check out single token embedding
sample_embed[0][0],sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.01160532, -0.01760237, -0.04184092, -0.04407846,  0.04875733,
        -0.00588049, -0.02924928, -0.02756965, -0.0269789 , -0.03189565,
        -0.03818621,  0.02887685,  0.02129215, -0.02272837,  0.00762744,
        -0.03016251, -0.01395427, -0.006113  , -0.02270945, -0.02742483,
        -0.03330179, -0.0162258 , -0.01706209,  0.03323879,  0.02217109,
        -0.02841805, -0.0079926 , -0.01442591, -0.00752914, -0.01763443,
        -0.04559371,  0.0060532 ,  0.04857535,  0.01777023, -0.02542138,
        -0.01740067,  0.03861291,  0.00753601,  0.00331046, -0.00218325,
         0.02259484,  0.04838986, -0.03849367,  0.01088142, -0.04144585,
        -0.0319282 ,  0.03775727,  0.00952493,  0.0487793 ,  0.03310759,
         0.02790171, -0.03960758,  0.01859156,  0.01923067, -0.015721  ,
         0.04946709,  0.01211246,  0.0360002 ,  0.04993967, -0.0336499 ,
         0.03703118,  0.04579041,  0.02611624, -0.04462754, -0.00028022,
  

### Modelling a text dataset

Once you've got your inputs and outputs prepared, it's a matter of figuring out which machine learning model to build in between them to bridge the gap.

Now that we've got a way to turn our text data into numbers, we can start to build machine learning models to model it.

To get plenty of practice, we're going to build a series of different models, each as its own experiment. We'll then compare the results of each model and see which one performed best.

More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data




### Model 0: Getting a baseline
As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll create a Scikit-Learn Pipeline using the TF-IDF (term frequency-inverse document frequency) formula to convert our words to numbers and then model them with the Multinomial Naive Bayes algorithm.

In [24]:
# Convert text into number
from sklearn.feature_extraction.text import TfidfVectorizer
# our model
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline

# create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),# convert words to number using tfidf
    ("clf",MultinomialNB())# model the text
])
# fit the pipleine to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score=model_0.score(val_sentences,val_labels)
# as we use .evaluate in tf for sklearn its .score
baseline_score*100

79.26509186351706

In [26]:
baseline_pred=model_0.predict(val_sentences)

baseline_pred[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [27]:
train_labels[:10]

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1])

### Creating an evaluation function for our model experiments

we could evaluate these as they are but since we're going to be evaluating several models in the same way going forward, let's create a helper function which takes an array of predictions and ground truth labels and computes the following:

* Accuracy
* Precision
* Recall
* F1-score
> 🔑 Note: Since we're dealing with a classification problem, the above metrics are the most appropriate. If we were working with a regression problem, other metrics such as MAE (mean absolute error) would be a better choice.

In [28]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def calculate_results(y_true,y_pred):
  # Calculate model accuracy
  model_accuracy=accuracy_score(y_true,y_pred)*100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision":model_precision*100,
                   "recall":model_recall*100,
                   "f1":model_f1*100
                   }
  return model_results

In [29]:
bline=calculate_results(y_true=val_labels, y_pred=baseline_pred)
bline

{'accuracy': 79.26509186351706,
 'precision': 81.11390004213173,
 'recall': 79.26509186351706,
 'f1': 78.6218975804955}

### Model 1: A Simple Dense Model 🚀

In [30]:
#  Creating tensorboard callback
from helper_functions import create_tensorboard_callback
SAVE_DIR="model_logs"

In [31]:
from tensorflow.keras import layers
input=layers.Input(shape=(1,),dtype=tf.string) # inputs are 1-dimensional
x=text_vectorizer(input) # convert strings into number
x=embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)
output=layers.Dense(1,activation="sigmoid")(x)
model_1=tf.keras.Model(input,output,name="model_1_dense")

In [32]:
# model_1.summary()
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230716-061236
Epoch 1/5
215/215 [==============================] - 33s 127ms/step - loss: 0.6115 - accuracy: 0.6935 - val_loss: 0.5343 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.4405 - accuracy: 0.8189 - val_loss: 0.4736 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3461 - accuracy: 0.8587 - val_loss: 0.4586 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2833 - accuracy: 0.8894 - val_loss: 0.4645 - val_accuracy: 0.7953
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2363 - accuracy: 0.9120 - val_loss: 0.4785 - val_accuracy: 0.7874



> use `GlobalAveragePooling1D` to layer before output layer if our data is 1D

In [33]:
model_1_pred=probs=model_1.predict(val_sentences)
model_1_pred.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [34]:
model_1_pred[:10]

array([[0.4435298 ],
       [0.7993007 ],
       [0.99790937],
       [0.12380005],
       [0.12031657],
       [0.9516888 ],
       [0.9102538 ],
       [0.9912497 ],
       [0.9716827 ],
       [0.272072  ]], dtype=float32)

In [35]:
tf.squeeze(tf.round(model_1_pred[:10]))==val_labels[:10]

<tf.Tensor: shape=(10,), dtype=bool, numpy=
array([ True, False,  True, False, False,  True,  True,  True,  True,
        True])>

In [36]:
words_in_voc = text_vectorizer.get_vocabulary()
len(words_in_voc),words_in_voc[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [37]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [38]:
# #  Get weight matrix of embedding layer
# embed_weight=model_1.get_layer("embedding").get_weights()[0]
# embed_weight

In [39]:
# # should be same size as vocab size and embedding dim
# print(embed_weight.shape)


`projector.tensorflow.org`
[word_embeddings guid](https://www.tensorflow.org/text/guide/word_embeddings)
* lets see how we can visualize embedding matrix token representationbn

In [40]:
#  import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(words_in_voc):
#   if num == 0:
#      continue # skip padding token
#   vec = embed_weight[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()

# # Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")

## *Recurrent Neural Network (RNN's)⛹*


* RNN's are the useful for sequence data.


* The premise of recurrent neueal network is to use the representation of previous input aid the representation of a later input.🔗


- [MIT Deep Learning Lecture on Recurrent Neural Networks](https://www.youtube.com/watch?v=SEnXr6v2ifU) - explains the background of recurrent neural networks and introduces LSTMs.

- [Understanding LSTMs by Chris Olah](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) - an in-depth (and technical) look at the mechanics of the LSTM cell, possibly the most popular RNN building block.


### Model 2 : LSTM
> LSTM = Long short term memory

* out structure of an RNN typically looks like this ⬇

```
Input (text) -> Tokenize -> embedding -> Layers (RNNs/dense) -> Output (label probability)
```

In [41]:
# LSTM model
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)

# print(x.shape)
x=layers.LSTM(64,return_sequences=True)(x)
# print(x.shape)

# print(x.shape)
x=layers.LSTM(64,return_sequences=True)(x)
x=layers.LSTM(64)(x)


outputs=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_2=tf.keras.Model(input,output,name="model_2_LSTM")

In [42]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
No

In [43]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [44]:
history=model_2.fit(train_sentences,
                    train_labels,
                    epochs=10,
                    validation_data=(val_sentences,val_labels),
                     callbacks=[create_tensorboard_callback(SAVE_DIR,'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230716-061321
Epoch 1/10
215/215 [==============================] - 11s 49ms/step - loss: 0.2005 - accuracy: 0.9270 - val_loss: 0.5138 - val_accuracy: 0.7861
Epoch 2/10
215/215 [==============================] - 2s 11ms/step - loss: 0.1719 - accuracy: 0.9372 - val_loss: 0.5427 - val_accuracy: 0.7887
Epoch 3/10
215/215 [==============================] - 2s 10ms/step - loss: 0.1481 - accuracy: 0.9456 - val_loss: 0.5746 - val_accuracy: 0.7835
Epoch 4/10
215/215 [==============================] - 2s 7ms/step - loss: 0.1290 - accuracy: 0.9553 - val_loss: 0.6136 - val_accuracy: 0.7848
Epoch 5/10
215/215 [==============================] - 2s 9ms/step - loss: 0.1135 - accuracy: 0.9583 - val_loss: 0.6488 - val_accuracy: 0.7835
Epoch 6/10
215/215 [==============================] - 2s 9ms/step - loss: 0.1023 - accuracy: 0.9632 - val_loss: 0.6863 - val_accuracy: 0.7782
Epoch 7/10
215/215 [==============================] - 2s 10ms/step - lo

In [45]:
model_2_pred_prob=model_2.predict(val_sentences)
model_2_pred_prob[:10]

24/24 [==============================] - 0s 2ms/step


array([[4.3123344e-01],
       [6.6302085e-01],
       [9.9969566e-01],
       [4.7636144e-02],
       [3.3173655e-04],
       [9.9143195e-01],
       [7.7191532e-01],
       [9.9999666e-01],
       [9.9999368e-01],
       [6.2317878e-01]], dtype=float32)

In [46]:
model_2_preds=tf.squeeze(tf.round(model_2_pred_prob))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [47]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [48]:
model_2_result=calculate_results(y_true=val_labels,
                                 y_pred=model_2_preds)
model_2_result

{'accuracy': 76.9028871391076,
 'precision': 77.06028054440213,
 'recall': 76.9028871391076,
 'f1': 76.69342344352704}

### Lets Try GRU 🌃
```
GRU is similar as LSTM but with less parameters
```


In [49]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [50]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [51]:
# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20230716-061350
Epoch 1/5
215/215 [==============================] - 15s 52ms/step - loss: 0.5263 - accuracy: 0.7284 - val_loss: 0.4566 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3174 - accuracy: 0.8692 - val_loss: 0.4903 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2165 - accuracy: 0.9183 - val_loss: 0.5571 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1541 - accuracy: 0.9460 - val_loss: 0.6203 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1172 - accuracy: 0.9606 - val_loss: 0.6001 - val_accuracy: 0.7690


In [52]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


((762, 1),
 array([[0.38547307],
        [0.9191302 ],
        [0.9969579 ],
        [0.16084224],
        [0.01584137],
        [0.988111  ],
        [0.75145245],
        [0.99661535],
        [0.99685955],
        [0.3750981 ]], dtype=float32))

In [53]:
# Convert prediction probabilities to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [54]:
# Calcuate model_3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.9028871391076,
 'precision': 76.91242349168364,
 'recall': 76.9028871391076,
 'f1': 76.78647757387915}

### Tensorflow Bi-direction LSTM(RNN) ✨
Look at us go! We've already built two RNN's with GRU and LSTM cells. Now we're going to look into another kind of RNN, the bidirectional RNN.

A standard RNN will process a sequence from left to right, where as a bidirectional RNN will process the sequence from left to right and then again from right to left.

Intuitively, this can be thought of as if you were reading a sentence for the first time in the normal fashion (left to right) but for some reason it didn't make sense so you traverse back through the words and go back over them again (right to left).

In practice, many sequence models often see and improvement in performance when using bidirectional RNN's.

However, this improvement in performance often comes at the cost of longer training times and increased model parameters (since the model goes left to right and right to left, the number of trainable parameters doubles).

Okay enough talk, let's build a bidirectional RNN.

Once again, TensorFlow helps us out by providing the tensorflow.keras.layers.Bidirectional class. We can use the Bidirectional class to wrap our existing RNNs, instantly making them bidirectional

In [55]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
# x=layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x) #layers.Bidirectional works with any LSTM
# print(x.shape)
x=layers.Bidirectional(layers.GRU(64))(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")


In [56]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              74496     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,354,625
Trainable params: 1,3

In [57]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [58]:
# Fit the model
model_4_history=model_4.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR, "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230716-061416
Epoch 1/5
215/215 [==============================] - 16s 51ms/step - loss: 0.1168 - accuracy: 0.9600 - val_loss: 0.9579 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0693 - accuracy: 0.9730 - val_loss: 0.8424 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0612 - accuracy: 0.9743 - val_loss: 0.9138 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0603 - accuracy: 0.9765 - val_loss: 0.9181 - val_accuracy: 0.7572
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0541 - accuracy: 0.9764 - val_loss: 1.0616 - val_accuracy: 0.7572


In [59]:
# convert pred probs to pred lables
model_4_pred_probs=model_4.predict(val_sentences)
# model_4_pred_probs[:10]
model_preds=tf.squeeze(tf.round(model_4_pred_probs))
model_preds[:10]

24/24 [==============================] - 1s 3ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [60]:
model_4_results=calculate_results(y_true=val_labels,
                                  y_pred=model_preds)
model_4_results

{'accuracy': 75.7217847769029,
 'precision': 75.7104469267424,
 'recall': 75.7217847769029,
 'f1': 75.60477242612443}

### Conv1D 🐛

In [61]:
embedding_test=embedding(text_vectorizer(['this is a test sentence']))
conv_1d=layers.Conv1D(filters=32,
                      kernel_size=5,
                      activation='relu',
                      padding='valid')
conv_1d_output=conv_1d(embedding_test)
max_pool=layers.GlobalMaxPool1D()
max_pool_output=max_pool(conv_1d_output)

In [62]:
# getting shape
embedding_test.shape,conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [63]:
# Create 1-D convolutional layer to model sequences

In [64]:
input=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Conv1D(filters=64,kernel_size=5,activation='relu',padding='valid')(x)
x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_5=tf.keras.Model(inputs,outputs,name="model_5_conv_1D")

model_5.compile(loss="binary_crossentropy",
                            optimizer=tf.keras.optimizers.Adam(),
                             metrics=["accuracy"])

model_5.summary()

Model: "model_5_conv_1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)             

In [65]:
model_5_history=model_5.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230716-061504
Epoch 1/5
215/215 [==============================] - 13s 46ms/step - loss: 0.1074 - accuracy: 0.9660 - val_loss: 0.9485 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0668 - accuracy: 0.9747 - val_loss: 1.1093 - val_accuracy: 0.7612
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0582 - accuracy: 0.9753 - val_loss: 1.1637 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0531 - accuracy: 0.9772 - val_loss: 1.2473 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0493 - accuracy: 0.9783 - val_loss: 1.2999 - val_accuracy: 0.7612


In [66]:
model_5_pred_pro=model_5.predict(val_sentences)
model_5_pred_pro[:10]

24/24 [==============================] - 0s 2ms/step


array([[2.1464747e-01],
       [6.4758664e-01],
       [9.9982589e-01],
       [5.8142170e-02],
       [5.5049571e-07],
       [9.9082464e-01],
       [9.9380869e-01],
       [9.9993730e-01],
       [1.0000000e+00],
       [9.6893811e-01]], dtype=float32)

In [67]:
model_5_preds=tf.squeeze(tf.round(model_5_pred_pro))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [68]:
# evaluate model_5 prediction
model_5_results=calculate_results(y_true=val_labels,
                                                        y_pred=model_5_preds)
model_5_results

{'accuracy': 76.11548556430446,
 'precision': 76.18883943071393,
 'recall': 76.11548556430446,
 'f1': 75.93763358258425}

### pre-trained sentence encoder ✈

For all of the previous deep learning models we've built and trained, we've created and used our own embeddings from scratch each time.

However, a common practice is to leverage pretrained embeddings through transfer learning. This is one of the main benefits of using deep models: being able to take what one (often larger) model has learned (often on a large amount of data) and adjust it for our own use case.

For our next model, instead of using our own embedding layer, we're going to replace it with a pretrained embedding layer.

More specifically, we're going to be using the Universal Sentence Encoder from TensorFlow Hub (a great resource containing a plethora of pretrained model resources for a variety of tasks).

> 🔑 Note: There are many different pretrained text embedding options on TensorFlow Hub, however, some require different levels of text preprocessing than others. Best to experiment with a few and see which best suits your use case.

In [70]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

The feature extractor model we're building through the eyes of an encoder/decoder* model.*

>🔑 Note: An encoder is the name for a model which converts raw data such as text into a numerical representation (feature vector), a decoder converts the numerical representation to a desired output.

As usual, this is best demonstrated with an example.

We can load in a TensorFlow Hub module using the hub.load() method and passing it the target URL of the module we'd like to use, in our case, it's "https://tfhub.dev/google/universal-sentence-encoder/4".

Let's load the Universal Sentence Encoder model and test it on a couple of sentences

In [74]:
import tensorflow_hub as hub
embed=hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
embed_sample=embed([sample_sentence,
                    "When you call the universal sentence encoder on a sentence, it turns it into numbers."])


embed_sample

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.01157028,  0.0248591 ,  0.02878048, ..., -0.00186124,
         0.02315826, -0.01485021],
       [ 0.03596688, -0.08579469, -0.01152738, ..., -0.03414334,
         0.02816022, -0.00878943]], dtype=float32)>

In [79]:
sentence_encoder_layer=hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                                                    input_shape=(),
                                                                    dtype=tf.string,
                                                                    trainable=False,
                                                                    name="USE"
                                                                    )

In [83]:
model_6=tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(1,activation='sigmoid')
],name='model_6_USE')
model_6.compile(loss='binary_crossentropy',
                            optimizer=tf.keras.optimizers.Adam(),
                            metrics=['accuracy']
                                )
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_9 (Dense)             (None, 1)                 513       
                                                                 
Total params: 256,798,337
Trainable params: 513
Non-trainable params: 256,797,824
_________________________________________________________________
